In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import os
import sys
sys.path.append("/home/armoraux/Python/pysteps_eval/github/pystepsval")
from importer import read_radar, read_netcdf
from processor import process_data_single_init_time

Pysteps configuration file found at: /home/armoraux/miniconda3/envs/xrVal/lib/python3.11/site-packages/pysteps/pystepsrc



In [2]:
data_dir = "/home/armoraux/Python/pysteps_eval/github/data"

In [3]:
pysteps_fp = os.path.join(data_dir, "pysteps/blended_nowcast_202509090000.nc")
pysteps_ds = read_netcdf(pysteps_fp)
pysteps_ds

Reading netCDF-file...done (0.00m)
Dataset is a valid xarray Dataset.
All required dimensions {'y', 'x', 'time'} are present.
'precip_intensity' variable is present in the dataset.


<xarray.Dataset> Size: 3GB
Dimensions:                  (y: 700, x: 700, ens_number: 24, time: 72)
Coordinates:
    lon                      (y, x) float64 4MB -0.9174 -0.9023 ... 8.983 8.997
    lat                      (y, x) float64 4MB 53.69 53.69 ... 47.42 47.42
  * x                        (x) float32 3kB 3.005e+05 3.015e+05 ... 9.995e+05
  * y                        (y) float32 3kB 9.995e+05 9.985e+05 ... 3.005e+05
  * ens_number               (ens_number) int64 192B 1 2 3 4 5 ... 21 22 23 24
  * time                     (time) datetime64[ns] 576B 2025-09-09T00:05:00 ....
    forecast_reference_time  datetime64[ns] 8B 2025-09-09
    leadtime                 (time) timedelta64[ns] 576B 00:05:00 ... 06:00:00
Data variables:
    precip_intensity         (ens_number, time, y, x) float32 3GB ...
Attributes:
    Conventions:  CF-1.7
    title:        pysteps-generated nowcast
    institution:  the pySTEPS community (https://pysteps.github.io)
    source:       pysteps
    history:      
    references:   
    comment:      
    projection:   +proj=lcc +lat_1=49.83333333333334 +lat_2=51.16666666666666...

In [4]:
radar_path_dict = {
    "root_path": os.path.join(data_dir, "radar"),
    "path_fmt": "%Y/%m/%d/best/comp/rate/qpe_edk/hdf",
    "fn_pattern": "%Y%m%d%H%M00.rad.best.comp.rate.qpe_edk",
    "fn_ext": "hdf",
    "importer_kwargs": {'accutime': 5.0, 'ignore_gain_offset': True}
    }
radar_ds = read_radar(pysteps_ds.time, radar_path_dict)
radar_ds

Reading RADAR data...2025-09-09T06:00:00.000000000
done (0.02m)
Dataset is a valid xarray Dataset.
All required dimensions {'y', 'x', 'time'} are present.
'precip_intensity' variable is present in the dataset.


<xarray.Dataset> Size: 149MB
Dimensions:           (time: 72, y: 700, x: 700)
Coordinates:
  * time              (time) datetime64[ns] 576B 2025-09-09T00:05:00 ... 2025...
  * x                 (x) float64 6kB 3.005e+05 3.015e+05 ... 9.995e+05
  * y                 (y) float64 6kB 9.995e+05 9.985e+05 ... 3.005e+05
    lon               (y, x) float64 4MB -0.9174 -0.9023 -0.8872 ... 8.983 8.997
    lat               (y, x) float64 4MB 53.69 53.69 53.69 ... 47.42 47.42 47.42
Data variables:
    precip_intensity  (time, y, x) float32 141MB nan nan nan nan ... nan nan nan

In [5]:
# Scoring parameters
thresholds = [0.1,0.5,1.0,5.0]
bin_edges = np.linspace(-0.000001, 1.000001, 11)
bins_x = np.arange(0.05, 1, 0.1)
fss_window_sizes = [5, 10, 30, 60]

scores_ds = process_data_single_init_time(
                pysteps_ds.isel(time=slice(0,10), ens_number=slice(0,4)), 
                radar_ds.isel(time=slice(0,10)),
                thresholds=thresholds, 
                bin_edges=bin_edges, 
                bins_x=bins_x, 
                fss_window_sizes=fss_window_sizes, 
                timing=True)

Dataset is a valid xarray Dataset.
All required dimensions {'y', 'x', 'time'} are present.
'precip_intensity' variable is present in the dataset.
Dataset is a valid xarray Dataset.
All required dimensions {'y', 'x', 'time'} are present.
'precip_intensity' variable is present in the dataset.
	All checks are done for runtime 2025-09-09T00:00:00.000000000, provided data conforms to expectations
	Elapsed time for prepocessing: 3.4200 seconds
	Done with all preprocessing for runtime: 2025-09-09T00:00:00.000000000


## Testing fortran FSS computation

In [6]:
from processor_test_fss_fortran import process_data_single_init_time

In [7]:
%%time

# Scoring parameters
thresholds = [0.1,0.5,1.0,5.0]
bin_edges = np.linspace(-0.000001, 1.000001, 11)
bins_x = np.arange(0.05, 1, 0.1)
fss_window_sizes = [5, 10, 30, 60]

scores_ds = process_data_single_init_time(
                pysteps_ds.isel(time=slice(0,24), ens_number=slice(0,12)), 
                radar_ds.isel(time=slice(0,24)),
                thresholds=thresholds, 
                bin_edges=bin_edges, 
                bins_x=bins_x, 
                fss_window_sizes=fss_window_sizes, 
                timing=False)

scores_ds = scores_ds.compute()

Dataset is a valid xarray Dataset.
All required dimensions {'y', 'x', 'time'} are present.
'precip_intensity' variable is present in the dataset.
Dataset is a valid xarray Dataset.
All required dimensions {'y', 'x', 'time'} are present.
'precip_intensity' variable is present in the dataset.
	All checks are done for runtime 2025-09-09T00:00:00.000000000, provided data conforms to expectations
	Done with all preprocessing for runtime: 2025-09-09T00:00:00.000000000
CPU times: user 2min 19s, sys: 3.6 s, total: 2min 22s
Wall time: 28.3 s


## Testing python (Daniel) FSS computation

In [8]:
from processor_test_fss_python import process_data_single_init_time

In [ ]:
%%time

scores2_ds = process_data_single_init_time(
                pysteps_ds.isel(time=slice(0,24), ens_number=slice(0,12)), 
                radar_ds.isel(time=slice(0,24)),
                thresholds=thresholds, 
                window_sizes=fss_window_sizes, 
                bin_edges=bin_edges, 
                bins_x=bins_x, 
                timing=False)

scores2_ds = scores2_ds.compute()

Dataset is a valid xarray Dataset.
All required dimensions {'y', 'x', 'time'} are present.
'precip_intensity' variable is present in the dataset.
Dataset is a valid xarray Dataset.
All required dimensions {'y', 'x', 'time'} are present.
'precip_intensity' variable is present in the dataset.
	All checks are done for runtime 2025-09-09T00:00:00.000000000, provided data conforms to expectations
	Done with all preprocessing for runtime: 2025-09-09T00:00:00.000000000
